In [1]:
import numpy as np
import matplotlib.pyplot as plt

def lorenz_attractor(sigma, rho, beta, x0, y0, z0, h, num_steps):
    
    # Criamos np.arrays de modo a otimizar o uso de listas
    # Operações mais rápidas entre os vetores
    x = np.zeros(num_steps)
    y = np.zeros(num_steps)
    z = np.zeros(num_steps)

    x[0], y[0], z[0] = x0, y0, z0

    for i in range(1, num_steps):
        k1_x = h * (sigma * (y[i-1] - x[i-1]))
        k1_y = h * (x[i-1] * (rho - z[i-1]) - y[i-1])
        k1_z = h * (x[i-1] * y[i-1] - beta * z[i-1])

        k2_x = h * (sigma * ((y[i-1] + 0.5 * k1_y) - (x[i-1] + 0.5 * k1_x)))
        k2_y = h * ((x[i-1] + 0.5 * k1_x) * (rho - (z[i-1] + 0.5 * k1_z)) - (y[i-1] + 0.5 * k1_y))
        k2_z = h * ((x[i-1] + 0.5 * k1_x) * (y[i-1] + 0.5 * k1_y) - beta * (z[i-1] + 0.5 * k1_z))

        k3_x = h * (sigma * ((y[i-1] + 0.5 * k2_y) - (x[i-1] + 0.5 * k2_x)))
        k3_y = h * ((x[i-1] + 0.5 * k2_x) * (rho - (z[i-1] + 0.5 * k2_z)) - (y[i-1] + 0.5 * k2_y))
        k3_z = h * ((x[i-1] + 0.5 * k2_x) * (y[i-1] + 0.5 * k2_y) - beta * (z[i-1] + 0.5 * k2_z))

        k4_x = h * (sigma * ((y[i-1] + k3_y) - (x[i-1] + k3_x)))
        k4_y = h * ((x[i-1] + k3_x) * (rho - (z[i-1] + k3_z)) - (y[i-1] + k3_y))
        k4_z = h * ((x[i-1] + k3_x) * (y[i-1] + k3_y) - beta * (z[i-1] + k3_z))

        x[i] = x[i-1] + (1/6) * (k1_x + 2*k2_x + 2*k3_x + k4_x)
        y[i] = y[i-1] + (1/6) * (k1_y + 2*k2_y + 2*k3_y + k4_y)
        z[i] = z[i-1] + (1/6) * (k1_z + 2*k2_z + 2*k3_z + k4_z)

    return x, y, z

# Arbitrando valores usuais
sigma = 10
rho = 28
beta = 8/3
x0, y0, z0 = 0, 1, 1.05
h = 0.01
num_steps = 100

x, y, z = lorenz_attractor(sigma, rho, beta, x0, y0, z0, h, num_steps)

# Guardando os dados em um dataset
import pandas as pd
df = pd.DataFrame({'X': x, 'Y': y, 'Z': z})


## Gráfico com plot RK, mexer em num_steps

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot(x, y, z)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('Atrator de Lorenz')

# plt.show()

In [2]:
display(df)

,X,Y,Z
0,0.000000,1.000000,1.050000
1,0.095105,1.003039,1.022849
2,0.182649,1.030512,0.997333
3,0.265581,1.080595,0.973428
4,0.346436,1.152210,0.951187
...,...,...,...
95,-9.537788,-10.261330,27.532698
96,-9.603472,-10.192146,27.774308
97,-9.655256,-10.101063,28.007867
98,-9.692420,-9.988976,28.230024


In [3]:
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import CubicSpline

# Criando splines cúbicas para os pontos (x, y, z)
spline_x = CubicSpline(df.index, df['X'])
spline_y = CubicSpline(df.index, df['Y'])
spline_z = CubicSpline(df.index, df['Z'])

fig = go.Figure()

# Dataset original
fig.add_trace(go.Scatter(x=df.index, y=df['X'], mode='markers', name='X', marker=dict(size=5)))
fig.add_trace(go.Scatter(x=df.index, y=df['Y'], mode='markers', name='Y', marker=dict(size=5)))
fig.add_trace(go.Scatter(x=df.index, y=df['Z'], mode='markers', name='Z', marker=dict(size=5)))

# trajetórias das splines cúbicas
fig.add_trace(go.Scatter(x=df.index, y=spline_x(df['X']), mode='lines', name='X_spline'))
fig.add_trace(go.Scatter(x=df.index, y=spline_y(df['Y']), mode='lines', name='Y_spline'))
fig.add_trace(go.Scatter(x=df.index, y=spline_z(df['Z']), mode='lines', name='Z_spline'))

fig.update_layout(
    title='Atrator de Lorenz - Visão Spline Cúbica',
    xaxis_title='t',
    yaxis_title='P(t)',
    legend=dict(
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='white',
        bordercolor='Black',
        borderwidth=2
    ),
    xaxis=dict(showgrid=True, gridcolor='LightGrey'),
    yaxis=dict(showgrid=True, gridcolor='LightGrey')
)

fig.show()

In [4]:
import numpy as np

# Definição do dataset em cada eixo
x = np.array(df.index)
y = np.array(df['X'])
z = np.array(df['Y'])

# coeficientes dos polinômios interpoladores usando MMQ
coeffs_x = np.polyfit(x, y, 3)
coeffs_y = np.polyfit(x, z, 3)
coeffs_z = np.polyfit(x, df['Z'], 3)

print(f'p(x) = {coeffs_x[0]}x^3 + {coeffs_x[1]}x^2 + {coeffs_x[2]}x + {coeffs_x[3]}\n')
print(f'p(y) = {coeffs_y[0]}x^3 + {coeffs_y[1]}x^2 + {coeffs_y[2]}x + {coeffs_y[3]}\n')
print(f'p(z) = {coeffs_z[0]}x^3 + {coeffs_z[1]}x^2 + {coeffs_z[2]}x + {coeffs_z[3]}')


Expressão do polinômio interpolador para X:
p(x) = 0.00015112173134043756x^3 + -0.029226080467680843x^2 + 1.385412878887583x + -8.17495142255034

Expressão do polinômio interpolador para Y:
p(y) = 0.0002898725333223482x^3 + -0.04761649038468331x^2 + 1.9018165558396305x + -8.316370542114292

Expressão do polinômio interpolador para Z:
p(z) = -3.648066137856111e-05x^3 + -0.004424719222436199x^2 + 1.0863863653085863x + -9.275098005104844


In [5]:
import numpy as np
import plotly.graph_objects as go

x_vals = np.linspace(df.index.min(), df.index.max(), 1000)

fig = go.Figure()

# Dataset Original
fig.add_trace(go.Scatter(x=df.index, y=df['X'], mode='markers', name='X', marker=dict(size=5)))
fig.add_trace(go.Scatter(x=df.index, y=df['Y'], mode='markers', name='Y', marker=dict(size=5)))
fig.add_trace(go.Scatter(x=df.index, y=df['Z'], mode='markers', name='Z', marker=dict(size=5)))

# Trajetórias MMQ
fig.add_trace(go.Scatter(x=x_vals, y=np.polyval(coeffs_x, x_vals), mode='lines', name='X_MMQ'))
fig.add_trace(go.Scatter(x=x_vals, y=np.polyval(coeffs_y, x_vals), mode='lines', name='Y_MMQ'))
fig.add_trace(go.Scatter(x=x_vals, y=np.polyval(coeffs_z, x_vals), mode='lines', name='Z_MMQ'))

fig.update_layout(
    title='Atrator de Lorenz - Visão MMQ',
    xaxis_title='t',
    yaxis_title='P(t)',
    legend=dict(
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='white',
        bordercolor='Black',
        borderwidth=2
    ),
    xaxis=dict(showgrid=True, gridcolor='LightGrey'),
    yaxis=dict(showgrid=True, gridcolor='LightGrey')
)

fig.show()


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

# Gerando as previsões para as splines cúbicas
y_pred_spline_x = spline_x(df['X'])
y_pred_spline_y = spline_y(df['Y'])
y_pred_spline_z = spline_z(df['Z'])

# Gerando as previsões para os polinômios interpoladores pelo MMQ
y_pred_mmq_x = np.polyval(coeffs_x, df.index)
y_pred_mmq_y = np.polyval(coeffs_y, df.index)
y_pred_mmq_z = np.polyval(coeffs_z, df.index)

# Calculando as métricas para as splines cúbicas
mae_spline_x = mean_absolute_error(df['X'], y_pred_spline_x)
mae_spline_y = mean_absolute_error(df['Y'], y_pred_spline_y)
mae_spline_z = mean_absolute_error(df['Z'], y_pred_spline_z)

rmse_spline_x = np.sqrt(mean_squared_error(df['X'], y_pred_spline_x))
rmse_spline_y = np.sqrt(mean_squared_error(df['Y'], y_pred_spline_y))
rmse_spline_z = np.sqrt(mean_squared_error(df['Z'], y_pred_spline_z))

evs_spline_x = explained_variance_score(df['X'], y_pred_spline_x)
evs_spline_y = explained_variance_score(df['Y'], y_pred_spline_y)
evs_spline_z = explained_variance_score(df['Z'], y_pred_spline_z)

# Calculando as métricas para os polinômios interpoladores pelo MMQ
mae_mmq_x = mean_absolute_error(df['X'], y_pred_mmq_x)
mae_mmq_y = mean_absolute_error(df['Y'], y_pred_mmq_y)
mae_mmq_z = mean_absolute_error(df['Z'], y_pred_mmq_z)

rmse_mmq_x = np.sqrt(mean_squared_error(df['X'], y_pred_mmq_x))
rmse_mmq_y = np.sqrt(mean_squared_error(df['Y'], y_pred_mmq_y))
rmse_mmq_z = np.sqrt(mean_squared_error(df['Z'], y_pred_mmq_z))

evs_mmq_x = explained_variance_score(df['X'], y_pred_mmq_x)
evs_mmq_y = explained_variance_score(df['Y'], y_pred_mmq_y)
evs_mmq_z = explained_variance_score(df['Z'], y_pred_mmq_z)

# Exibindo os resultados
print('Métricas para Splines Cúbicas:')
print(f'MAE X: {mae_spline_x:.4f}, RMSE X: {rmse_spline_x:.4f}, EVS X: {evs_spline_x:.4f}')
print(f'MAE Y: {mae_spline_y:.4f}, RMSE Y: {rmse_spline_y:.4f}, EVS Y: {evs_spline_y:.4f}')
print(f'MAE Z: {mae_spline_z:.4f}, RMSE Z: {rmse_spline_z:.4f}, EVS Z: {evs_spline_z:.4f}')

print('\nMétricas para MMQ:')
print(f'MAE X: {mae_mmq_x:.4f}, RMSE X: {rmse_mmq_x:.4f}, EVS X: {evs_mmq_x:.4f}')
print(f'MAE Y: {mae_mmq_y:.4f}, RMSE Y: {rmse_mmq_y:.4f}, EVS Y: {evs_mmq_y:.4f}')
print(f'MAE Z: {mae_mmq_z:.4f}, RMSE Z: {rmse_mmq_z:.4f}, EVS Z: {evs_mmq_z:.4f}')


Métricas para Splines Cúbicas:
MAE X: 6.1282, RMSE X: 7.6759, EVS X: 0.2600
MAE Y: 8.9521, RMSE Y: 10.0546, EVS Y: 0.2904
MAE Z: 12.3681, RMSE Z: 16.0041, EVS Z: 0.4837

Métricas para MMQ:
MAE X: 3.4642, RMSE X: 4.3315, EVS X: 0.7553
MAE Y: 5.0671, RMSE Y: 6.3862, EVS Y: 0.6935
MAE Z: 6.9720, RMSE Z: 8.4984, EVS Z: 0.6637
